Code for generating names, starting with the language model.

In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
import html
import numpy as np

class CharacterEncoder(BaseEstimator, TransformerMixin):
    '''
    Transform a string into context and target character sequence numbers, using the ordinal
    value of each character.
    
    '''
    def __init__(self, context_length=16, maximum_ordinal=2**16):
        '''
        Parameters
        ----------
        context_length : int
            This number of characters will be used as a context to predict future characters.
        maximum_ordinal : int
            Limit total memory use in case you run into very high unicode characters.
        '''
        self.context_length = context_length
        self.maximum_ordinal = maximum_ordinal
        
    def fit(self, strings, **kwargs):
        '''
        No need to fit.
        '''
        return self

    def transform(self, strings):
        '''
        Transform an iterable source of strings into a dense matrix
        of character identifiers.
        
        Each sample will be a string snippet of context_length characters.
        
        Parameters
        ----------
        strings : iterable
            An iterable of source strings.
       
       Returns
        -------
        (np.ndarray, np.ndarray)
            A tuple (X, Y) 2 dimensional [sample_index, character], with a 32 bit character identifier, and
            a one dimensional [sample_index] with a 32 bit character identifier to predict.
        '''
        # forgive passing a single string
        if type(strings) is str:
            strings = [strings]
        # buffer up contexts and targets, we'll be predicting target characters
        # from context strings
        contexts = []
        targets = []
        for i, string in enumerate(strings):
            # lowercase and stripped of leading whitespace, makes the model more compact
            string = string.lower().strip()
            # null character termination for each string
            string += chr(0)
            for j in range(0, len(string) - self.context_length):
                contexts.append(string[j:j + self.context_length])
                targets.append(string[j + self.context_length])
        # blocks of memory to hold character ordinals
        X = np.zeros((len(contexts), self.context_length), dtype=np.int32)
        Y = np.zeros(len(targets), dtype=np.int32)
        # numerical encoding of character values
        for i, context in enumerate(contexts):
            for j, character in enumerate(context):
                X[i, j] = min(ord(character), self.maximum_ordinal)
        for i, character in enumerate(targets):
            Y[i] = min(ord(character), self.maximum_ordinal)
        return X, Y


In [2]:
class CharacterLanguageModelVectorizer(BaseEstimator, TransformerMixin):
    '''
    Base language model uses a CharacterEncoder to create character ordinals
    and then applies a transformation in order to create vectors.
    '''
    def __init__(self, context_length=16, maximum_ordinal=2**16):
        '''
        Parameters
        ----------
        context_length : int
            This number of characters will be used as a context to predict future characters.
        maximum_ordinal : int
            Limit total memory use in case you run into very high unicode characters.
        '''
        self.sequencer = CharacterEncoder(context_length, maximum_ordinal)
    
    def fit(self, strings):
        '''
        Nothing to fit.
        '''
        return self
    
    def transform(self, strings):
        '''
        Transform strings into a dense (X, Y) pairing.
        
        Parameters
        ----------
        strings : iterable
            An iterable of source strings.
       
       Returns
        -------
        (np.ndarray, np.ndarray)
            A tuple (X, Y) three dimensional [sample_index, character_index, one_hot] context X and
            a two dimensional [sample_index, one_hot] target Y.
        '''
        # character sequence numbers
        X, Y = self.sequencer.transform(strings)
        # one hot context encoding
        x = np.zeros((X.shape[0], self.sequencer.context_length, self.sequencer.maximum_ordinal), dtype=np.bool)
        y = np.zeros((Y.shape[0], self.sequencer.maximum_ordinal), dtype=np.bool)
        for i, context in enumerate(X):
            for t, char in enumerate(context):
                x[i, t, char] = 1
        for i, target in enumerate(Y):
            y[i, target] = 1
        return x, y
    
    def inverse_transform(self, X):
        '''
        Given a matrix of one hot encodings, reverse the transformation and return a matrix of characters.
        '''
        ordinals = X.argmax(-1).astype(np.int32)
        decode = np.vectorize(chr)
        return ''.join(decode(ordinals))

Now we can build up a recurrent neural network to learn a language model.

In [3]:
from keras.models import Sequential
from keras.layers import LSTM, CuDNNLSTM, Dropout, Dense, Reshape, BatchNormalization

class RecurrentLanguageModel(BaseEstimator):
    '''
    Create a language model with a neural network and normalized character encoding.
    '''

    def __init__(self, vectorizer, hidden_layers=256, gpu_optimized=False):
        '''
        Parameters
        ----------
        vectorizer : transformer
            Object to transform input strings into numerical encodings.
        hidden_layers : int
            Size of the model's hidden layer, controls complexity.
        gpu_optimized : bool
            If True, use special code in keras to boost performance.
        '''
        self.hidden_layers = hidden_layers
        self.gpu_optimized = gpu_optimized
        self.vectorizer = vectorizer

    def fit(self, strings, epochs=256, batch_size=256):
        '''
        Create and fit a model to the passed in strings.
        
        Parameters
        ----------
        strings : iterable
            An iterable source of string text.
        '''
        if self.gpu_optimized:
            RNN = CuDNNLSTM
        else:
            RNN = LSTM
        X, Y = self.vectorizer.transform(strings)
        self.X = X
        self.Y = Y
        self.model = model = Sequential()
        # input shape is represented as the shape of a single batch entry
        model.add(RNN(self.hidden_layers, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
        model.add(BatchNormalization())
        model.add(RNN(self.hidden_layers))
        model.add(BatchNormalization())
        model.add(Dense(self.hidden_layers, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dense(self.hidden_layers, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dense(Y.shape[1], activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam')
        model.fit(X, Y, epochs=epochs, batch_size=batch_size)

Using TensorFlow backend.
/home/wballard/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Run this model with cities. I'm using a GPU - and very much recommend you do so! You can set the optimization to False if you need to use a CPU.

City names aren't sentences, so we need to use a relatively short context length.

In [4]:
vectorizer = CharacterLanguageModelVectorizer(context_length=3, maximum_ordinal=128)
model = RecurrentLanguageModel(vectorizer, hidden_layers=512, gpu_optimized=True)
with open('cities.txt', encoding='utf8') as cities:
    model.fit(cities.readlines())

Epoch 1/256
145301/145301 [==============================] - 298s 2ms/step - loss: 2.2348
Epoch 2/256
145301/145301 [==============================] - 10s 70us/step - loss: 1.8655
Epoch 3/256
145301/145301 [==============================] - 10s 68us/step - loss: 1.8010
Epoch 4/256
145301/145301 [==============================] - 10s 68us/step - loss: 1.7617
Epoch 5/256
145301/145301 [==============================] - 10s 71us/step - loss: 1.7344
Epoch 6/256
145301/145301 [==============================] - 10s 67us/step - loss: 1.7123
Epoch 7/256
145301/145301 [==============================] - 10s 69us/step - loss: 1.6947
Epoch 8/256
145301/145301 [==============================] - 10s 66us/step - loss: 1.6789
Epoch 9/256
145301/145301 [==============================] - 10s 67us/step - loss: 1.6644
Epoch 10/256
145301/145301 [==============================] - 10s 67us/step - loss: 1.6545
Epoch 11/256
145301/145301 [==============================] - 10s 67us/step - loss: 1.6426
Epoch 12

145301/145301 [==============================] - 10s 67us/step - loss: 1.4964
Epoch 92/256
145301/145301 [==============================] - 10s 68us/step - loss: 1.4966
Epoch 93/256
145301/145301 [==============================] - 10s 67us/step - loss: 1.4960
Epoch 94/256
145301/145301 [==============================] - 10s 66us/step - loss: 1.4960
Epoch 95/256
145301/145301 [==============================] - 10s 68us/step - loss: 1.4949
Epoch 96/256
145301/145301 [==============================] - 10s 67us/step - loss: 1.4944
Epoch 97/256
145301/145301 [==============================] - 10s 67us/step - loss: 1.4945
Epoch 98/256
145301/145301 [==============================] - 10s 67us/step - loss: 1.4943
Epoch 99/256
145301/145301 [==============================] - 10s 67us/step - loss: 1.4940
Epoch 100/256
145301/145301 [==============================] - 10s 66us/step - loss: 1.4937
Epoch 101/256
145301/145301 [==============================] - 10s 68us/step - loss: 1.4934
Epoch 102/

145301/145301 [==============================] - 10s 67us/step - loss: 1.4825
Epoch 181/256
145301/145301 [==============================] - 10s 66us/step - loss: 1.4826
Epoch 182/256
145301/145301 [==============================] - 10s 65us/step - loss: 1.4815
Epoch 183/256
145301/145301 [==============================] - 10s 67us/step - loss: 1.4820
Epoch 184/256
145301/145301 [==============================] - 10s 66us/step - loss: 1.4824
Epoch 185/256
145301/145301 [==============================] - 10s 67us/step - loss: 1.4820
Epoch 186/256
145301/145301 [==============================] - 10s 66us/step - loss: 1.4818
Epoch 187/256
145301/145301 [==============================] - 9s 65us/step - loss: 1.4814
Epoch 188/256
145301/145301 [==============================] - 10s 67us/step - loss: 1.4824
Epoch 189/256
145301/145301 [==============================] - 10s 66us/step - loss: 1.4821
Epoch 190/256
145301/145301 [==============================] - 10s 66us/step - loss: 1.4819
Epo

Model converges relatively quickly -- more data or more training could be helpful.

And now for the interesting part -- generation. This uses a random seed to kick things off, and then pulls out characters until we hit a terminator.

In [8]:
class NameLanguageModelGenerator():
    '''
    Given a language model, generate new name strings given a seed of your own design.
    '''
    
    def __init__(self, language_model):
        '''
        Parameters
        ----------
        language_model
            A trained language model used to generate predictions.
        '''
        self.language_model = language_model
        
    def generate(self, seed, max_length=32):
        '''
        Parameters
        ----------
        seed : str
            A string to bootstrap generation.
        max_length: int
            A guard value to prevent looping forever.
        '''
        assert(len(seed) >= self.language_model.vectorizer.sequencer.context_length)
        # start appending after this number of context iterations
        append_after = len(seed) - self.language_model.vectorizer.sequencer.context_length
       
        # build up the result buffer here, adding on to our passed seed
        result = seed
        for i in range(0, max_length):
            X, _ = self.language_model.vectorizer.transform([seed])
            # only need the very first sample, then keep iterating
            try:
                next_character = self.language_model.vectorizer.inverse_transform(self.language_model.model.predict(X))[i]
            except IndexError:
                # when we hit a null character, it is time to exit
                break
            # keep expanding the seed with each generated character
            seed += next_character
            # and save off any results, expanding the seed string
            if i >= append_after:
                result += next_character
        return result.capitalize()

In a sense, this is a kind of a machine learning made up autocomplete, we'll start with a few characters and see what it tacks on!

In [9]:
NameLanguageModelGenerator(model).generate('Sam')

'Samont'